# Post-wildfire vegetation regrowth analysis
Author: Stepan Bryleev

### Normalized Burn Ratio (NBR)

In this Jupyter Notebook there is a JavaScript code for calculating and displaying Normalized Burn Ratio (NBR) index values for the study area in GEE using NEON reflectance data.

* NBR uses the spectral response of burned vegetation versus healthy vegetation to identify burned areas.
* $NBR=\frac{(NIR-SWIR)}{(NIR+SWIR)}$
* The difference normalized burn ratio (dNBR) is the difference between the pre-fire NBR and the post-fire NBR.

```js
// ====================================================== //
// This part of the project is for creating GRSM NBR maps //
// ====================================================== //

// Specify center location for GRSM
var SiteCenterPoint = ee.Geometry.Point([-83.5, 35.7]);

// Center the map on our area of interest and set zoom level
Map.setCenter(-83.5, 35.7, 12);

var study_polygon = /* color: #98ff00 */ee.Geometry.Polygon(
        [[[-83.5425714556907, 35.701602862979236],
          [-83.5422281329368, 35.690310616971246],
          [-83.53879490539774, 35.63828989061714],
          [-83.5037759844993, 35.615406781430224],
          [-83.48592430887022, 35.61938453875363],
          [-83.47459355041727, 35.627546807436595],
          [-83.47922808124717, 35.65117980654369],
          [-83.47738302453864, 35.682051678012314],
          [-83.4774257678571, 35.69326203209512],
          [-83.47751235961914, 35.71456632200802],
          [-83.47837061486136, 35.7182597023649],
          [-83.48043060302734, 35.72055934657099],
          [-83.53656387329102, 35.70494866628485]]]);

// Define the GRSM perimeter variable
var grsm_boundary = ee.FeatureCollection('projects/ee-stbr4432/assets/grsm_polygon');

// Define the fire perimeter variable
var fire_perimeter = ee.FeatureCollection('projects/ee-stbr4432/assets/chimney_tops_perim');
var fireBoundGeom = fire_perimeter.geometry();

// Apply the intersection method to the Polygon object.
var polygonIntersection = fireBoundGeom.intersection(grsm_boundary);
print('polygon intersection',polygonIntersection);

// Create study area.
var studyArea = study_polygon.intersection(polygonIntersection);

// Print polygon area in square kilometers.
print('Study area square: ', studyArea.area().divide(1000 * 1000));

// Print polygon perimeter length in kilometers.
print('Study area perimeter: ', studyArea.perimeter().divide(1000));


// === Create a function to check observation dates === //

function dates_check (image, greenRange, year) {
  // Define Acquisition_Dates
  var grsmDates = image.select(['Acquisition_Date']);
  var StartDate = grsmDates.reduceRegion({reducer: ee.Reducer.min(),maxPixels: 1e10});
  var EndDate = grsmDates.reduceRegion({reducer: ee.Reducer.max(),maxPixels: 1e10});
  // Define start and end dates
  var date_min = StartDate.get("Acquisition_Date");
  var date_max = EndDate.get("Acquisition_Date");
  // Format start and end dates
  var date_min_str = ee.String(date_min).replace('E7','').replace('2.','2');
  var date_max_str = ee.String(date_max).replace('E7','').replace('2.','2');
  
  var date_min_cat = ee.String(date_min_str.slice(0, 4).cat('-').cat(date_min_str.slice(4, 6)
                                                       .cat('-')).cat(date_min_str.slice(6, 8)));
  var date_max_cat = ee.String(date_max_str.slice(0, 4).cat('-').cat(date_max_str.slice(4, 6)
                                                       .cat('-')).cat(date_max_str.slice(6, 8)));

  var format_min_date = ee.Date(date_min_cat);
  var format_max_date = ee.Date(date_max_cat);
  // Define observation date range
  var obsRange = ee.DateRange(format_min_date, format_max_date);

  // Check observation date range
  var check = ee.Algorithms.If(greenRange.contains(obsRange),
    'Observation dates '+ year +' are within the summer peak greenness period.',
    'CAUTION! Observation dates '+ year +' are NOT within the summer peak greenness period!');
  print(check);
  
}


// ===== Create a cloud masking function ===== //

// Define a function
function neon_mask(image) {
  // Create a single band Weather Quality QA layer & Mask out cloudy pixels from the image
  var weather_qa = image.select(['Weather_Quality_Indicator']);
  var clear_qa = weather_qa.gt(0);
  var nis_image_clear = image.updateMask(clear_qa);
  return nis_image_clear;
}


// === Create a function to calculate and display NBR image with a date === //

// NBR = (NIR – SWIR) / (NIR + SWIR)
// NIR = NIS Band B097
// SWIR = NIS Band B365 (2206 nm)

// Define a function and bands of interest
function normdiff_nbr(image, date) {
  // NIR = NIS Band B097, SWIR = NIS Band B365
  var nis_normdiff = image.normalizedDifference(["B097", "B365"]);
  var viz_params = {min:-1, max:1, palette:['brown', 'DimGray', 'lightgreen', 'darkgreen']};
  Map.addLayer(nis_normdiff, viz_params, "GRSM NBR " + date, 0);
  
  return nis_normdiff;
}


// === Create a function to calculate dNBR image multipied by 1000 === //

// Define a function
function calc_dNBR(pre_fire_im, post_fire_im) {
  // Subtract images and multiply by 1000
  var dNBR_unscaled = pre_fire_im.subtract(post_fire_im);
  var dNBR_image = dNBR_unscaled.multiply(1000);

  return dNBR_image;
}


// === Create a function to plot histogram charts === //

// Define a function
function plot_hist(diff_image, studyArea, scale, title) {
  var unscaled_im = diff_image.divide(1000);
  var diff_hist = ui.Chart.image.histogram({image: unscaled_im, region: studyArea, scale: scale})
      .setOptions({title: title,
            hAxis: {title: 'dNBR Value',titleTextStyle: {italic: false, bold: true},},
            vAxis: {title: 'Count', titleTextStyle: {italic: false, bold: true}},});
            
  print(diff_hist);
}

// DEFINE DATES OF INTEREST:
var date_2016 = '2016-06-08';
var date_2017 = '2017-10-05'; 
var date_2021 = '2021-06-18';

//  DATA COLLECTION: 'projects/neon-prod-earthengine/assets/DP3-30006-001'

// ======= Read 2016 data ======= //

// Read in the 2016 SDR Image for GRSM
var NISimage_2016 = ee.Image('projects/neon-prod-earthengine/assets/DP3-30006-001/2016_GRSM_2_SDR');

// Define 2016 summer greenness period 
var greenRange_2016 = ee.DateRange('2016-05-15', '2016-08-15');
// Check 2016 observation dates 
dates_check(NISimage_2016, greenRange_2016, '2016');

// Apply masking function to 2016 image and clip
var nis_image_2016_masked = neon_mask(NISimage_2016.clip(studyArea));


// ======= Read 2017 data ======= //

// Read in the 2017 SDR Image for GRSM
var NISimage_2017 = ee.Image('projects/neon-prod-earthengine/assets/DP3-30006-001/2017_GRSM_3_SDR');
  
// Define 2017 summer greenness period 
var greenRange_2017 = ee.DateRange('2017-05-15', '2017-08-15');

// Check 2017 observation dates 
dates_check(NISimage_2017, greenRange_2017, '2017');

// Apply masking function to 2017 image and clip
var nis_image_2017_masked = neon_mask(NISimage_2017.clip(studyArea));


// ======= Read 2021 data ======= //

// Read in the 2021 SDR Image for GRSM
var NISimage_2021 = ee.Image('projects/neon-prod-earthengine/assets/DP3-30006-001/2021_GRSM_5_SDR');

// Define 2021 summer greenness period 
var greenRange_2021 = ee.DateRange('2021-05-15', '2021-08-15');

// Check 2016 observation dates 
dates_check(NISimage_2021, greenRange_2021, '2021');

// Apply masking function to 2021 image and clip
var nis_image_2021_masked = neon_mask(NISimage_2021.clip(studyArea));


// ======= Calculate and plot NBR for each date of interest ======= //

var pre_fire_nbr_2016 = normdiff_nbr(nis_image_2016_masked, date_2016);
var post_fire_nbr_2017 = normdiff_nbr(nis_image_2017_masked, date_2017);
var post_fire_nbr_2021 = normdiff_nbr(nis_image_2021_masked, date_2021);


// ======= Calculate dNBR images of difference ======= //

var dNBR_2017_2016 = calc_dNBR(pre_fire_nbr_2016, post_fire_nbr_2017);
var dNBR_2021_2016 = calc_dNBR(pre_fire_nbr_2016, post_fire_nbr_2021);


// ======= Create dNBR Classification and add dNBR images to the map ======= //

// SLD = Styled Layer Descriptor

// Define an SLD style of discrete intervals to apply to dNBR images
var sld_intervals =
  '<RasterSymbolizer>' +
    '<ColorMap type="intervals" extended="false" >' +
      '<ColorMapEntry color="#ffffff" quantity="-500" label="-500"/>' +
      '<ColorMapEntry color="#7a8737" quantity="-250" label="-250" />' +
      '<ColorMapEntry color="#acbe4d" quantity="-100" label="-100" />' +
      '<ColorMapEntry color="#0ae042" quantity="100" label="100" />' +
      '<ColorMapEntry color="#fff70b" quantity="270" label="270" />' +
      '<ColorMapEntry color="#ffaf38" quantity="440" label="440" />' +
      '<ColorMapEntry color="#ff641b" quantity="660" label="660" />' +
      '<ColorMapEntry color="#a41fd6" quantity="2000" label="2000" />' +
    '</ColorMap>' +
  '</RasterSymbolizer>';

// Add dNBR images to the map using both the color ramp and interval schemes
Map.addLayer(dNBR_2017_2016.sldStyle(sld_intervals), {}, '2017/2016 dNBR classified');
Map.addLayer(dNBR_2021_2016.sldStyle(sld_intervals), {}, '2021/2016 dNBR classified');


// =======   ADD A dNBR LEGEND  ======== //

// Create legend panel and set its position
var legend = ui.Panel({
  style: {
    position: 'bottom-left',
    padding: '8px 20px'
  }});
 
// Create legend title
var legendTitle = ui.Label({
  value: 'dNBR Classes',
  style: {fontWeight: 'bold',
    fontSize: '18px',
    margin: '0 0 6px 0',
    padding: '0'
    }});
 
// Add the title to the panel
legend.add(legendTitle);
 
// Creates and styles 1 row of the legend.
var makeRow = function(color, name) {
 
      // Create the label that is actually the colored box.
      var colorBox = ui.Label({
        style: {
          backgroundColor: '#' + color,
          // Use padding to give the box height and width.
          padding: '8px',
          margin: '0 0 4px 0'
        }});
 
      // Create the label filled with the description text.
      var description = ui.Label({
        value: name,
        style: {margin: '0 0 4px 6px'}
      });
 
      // return the panel
      return ui.Panel({
        widgets: [colorBox, description],
        layout: ui.Panel.Layout.Flow('horizontal')
      })};
 
//  Palette with the colors
var palette =['7a8737', 'acbe4d', '0ae042', 'fff70b', 'ffaf38', 'ff641b', 'a41fd6', 'ffffff'];
 
// name of the legend
var names = ['Enhanced Regrowth, High','Enhanced Regrowth, Low','Unburned', 'Low Severity',
'Moderate-low Severity', 'Moderate-high Severity', 'High Severity', 'NA'];
 
// Add color and and names to a legend panel
for (var i = 0; i < 8; i++) {
  legend.add(makeRow(palette[i], names[i]));
  }  
 
// Add legend to the map
Map.add(legend);


// ======= Display Boundaries ======= //

// Display the fire boundary
Map.addLayer(fire_perimeter.style({width: 2,
                                  color: "red",
                                  fillColor: "#00000000"}),{},"Fire Boundary", 1);


// ======= Plot histogram charts ======= //

// Plot histogram charts for each dNBR image in the console
plot_hist(dNBR_2017_2016, studyArea, 50, 'dNBR 2016/2017');
plot_hist(dNBR_2021_2016, studyArea, 50, 'dNBR 2016/2021');

```